In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 30)
import requests
%run "00_common_res.ipynb"
import os
%run func/df_to_csv.ipynb

In [ ]:
"""
steps
1. get data from S3
2. wheater data transfommations
3. taxi data transfommations
4. update payment types table
5. update companys table
6. update taxi data with company and payment types ids (replace strings with id from latest tables)
7-8. upload taxi and weather data to s3
9. upload new company and payment types table to s3
"""

### get data

In [ ]:
# get 1 day trip data from protal by API
url_chportal_taxi_api = (f"https://data.cityofchicago.org/resource/ajtu-isnz.json?"
    f"$where=trip_start_timestamp >= '{formatted_date}T00:00:00' "
    f"AND trip_start_timestamp <= '{formatted_date}T23:59:59'&$limit=30000")
res_chportal_api = requests.get(url_chportal_taxi_api)
# format to json
js_taxi_data = res_chportal_api.json()
# convert json format data to dataFrame
df_taxi_data =  pd.DataFrame (js_taxi_data)

### Trasfomation part

In [ ]:
# drop empty columns
df_taxi_data.drop(["pickup_census_tract", "dropoff_census_tract"], axis=1, inplace=True)
df_taxi_data.drop(["pickup_centroid_location", "dropoff_centroid_location"], axis=1, inplace=True)

In [ ]:
# drop rows that have missings
df_taxi_data.dropna(inplace=True) 

In [ ]:
# renaming cols
di_col_old_new = {
    "pickup_community_area": "pickup_community_area_id", 
    "dropoff_community_area": "dropoff_community_area_id"
    }
df_taxi_data.rename (columns=di_col_old_new, inplace=True)

In [ ]:
# create helper column
df_taxi_data["datetime_for_weather"] = pd.to_datetime(df_taxi_data["trip_start_timestamp"])

In [ ]:
df_taxi_data.head()

#### taxi data transformation function

In [ ]:
def taxi_data_transformations(df_taxi_data):
    """ taxi_data_transformations
    ver: v1.0
    Params:
        df_taxi_data (df): taxi data
    Returns:
        df: cleaned a trasform taxi data
    requirements:
        module(s): pandas
    """
    # drop empty columns
    df_taxi_data.drop(["pickup_census_tract", "dropoff_census_tract", "pickup_centroid_location", "dropoff_centroid_location"], axis=1, inplace=True)
    # drop rows that have missings
    df_taxi_data.dropna(inplace=True) 
    # renaming cols
    di_col_old_new = {
        "pickup_community_area": "pickup_community_area_id", 
        "dropoff_community_area": "dropoff_community_area_id"
        }
    df_taxi_data.rename (columns=di_col_old_new, inplace=True)
    # create helper column
    df_taxi_data["datetime_for_weather"] = pd.to_datetime(df_taxi_data["trip_start_timestamp"])
    
    return df_taxi_data